Goal of analysis script is to take csv file of acoustic startle data (generated from the 'concatenate' function of SR Labs AS GUI) and analyze each AS block for each animal in the file. Two different blocks (with varrying numbers of trials). Result is excel file with a tab for each block.

In [1]:
import numpy as np

Use tkinter to open filedialog prompting user to select file to analyze.

In [2]:
from tkinter import filedialog
from tkinter import *
 
root = Tk()
root.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"),("all files","*.*")))
print (root.filename)

C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Excel_files/Behaviors/AcousticStartle/original/VNS/Round 3/AS VNS 84d 190410.csv


In [3]:
fulldata = np.genfromtxt(root.filename, delimiter=",", dtype="U8")

print(fulldata[0])
print(len(fulldata))

['4/10/201' '09:48:41' ' AbbieHa' '124' ' No Entr' 'No Entry' '       1'
 '110dBAS ' '      1 ' '1' '15' '83' '18' '14']
1911


Generate list of animal numbers for iteration use **animal_numbers[i]

In [4]:
animal_numbers = []
for row in fulldata:
    if row[3] in animal_numbers:
        animal_numbers
    else: 
        animal_numbers.append(row[3])
        
print(animal_numbers)

['124', '125', '126', '112', '114', '140', '115', '116', '117', '127', '128', '131', '118', '119', '120', '133', '134', '135', '136', '138', '109']


Parse data for animals into separate data sets (1 per animal) for iteration use **animal_data[animal_numbers[i]]

In [5]:
data = {}
i = 0
while i < len(animal_numbers):
    print(i)
    animal = []
    for row in fulldata:
        if row[3] == animal_numbers[i]:
            animal.append(row)
    data[animal_numbers[i]] = animal
    i = i + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


Find average basal startle level (assessed once per each two min break between blocks). Basal startle level = animal_numbers[0][50][7]. Search for basal startle level parameter by searching for row[7] (parameter name).

In [6]:
i = 0
twobreak_final = {}
while i < len(animal_numbers):
    print(i)
    sum = 0
    for row in data[animal_numbers[i]]:
        if row[7] == data[animal_numbers[0]][50][7]:
            sum = sum + np.int(row[11])
    ave = sum/
    twobreak_final[animal_numbers[i]] = ave
    i = i + 1  
    

SyntaxError: invalid syntax (<ipython-input-6-906d7f54691a>, line 9)

Block 1 is 50 trials of the same parameter (110dB). To assess habituation across trials, first bin by 10 trials and then put each animal's data into an array. 

In [7]:
i = 0
block1ave_final = {}
while i < len(animal_numbers):
    print(i)
    j = 0
    ave_list = []
    k = 0
    sum = 0
    while j < 50:
        while k < 10:
            sum = sum + np.int((data[animal_numbers[i]][j][11]))
            k = k + 1
            j = j + 1
        ave = sum/10
        ave_list.append(ave)
        k = 0
        sum = 0
    block1ave_final[animal_numbers[i]] = ave_list
    i = i + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


Compute average startle of first 5 block 1 trials to use to calculate block 2 PPI

In [8]:
i = 0
block1ave_firstfive = {}
while i < len(animal_numbers):
    print(i)
    j = 0
    ave_list = []
    k = 0
    sum = 0
    while j < 5:
        while k < 5:
            sum = sum + np.int((data[animal_numbers[i]][j][11]))
            k = k + 1
            j = j + 1
        ave = sum/5
        ave_list.append(ave)
        k = 0
        sum = 0
    block1ave_firstfive[animal_numbers[i]] = ave_list
    i = i + 1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


Block 2 contains multiple trials of multiple different parameters (pesudo-randomly assigned to trials). General strategy is to generate list of parameters for the block. Find average of each parameter for each animal. After finding average of each parameter, need to also compute PPI.

Generate a list of block_2 parameters

In [9]:
j = 0
block2_param = []
for row in data[animal_numbers[1]][51:91]:
    if row[7] in block2_param:
        j = j + 1
    else: 
        block2_param.append(row[7])
        j = j + 1
    
print(block2_param)

['200msAS ', '20msAS  ', '100msAS ', '50msAS  ', '2secAS  ', '1000msAS', '500msAS ', '10msAS  ']


Block 2 assesses PPI and is 40 trials of 8 different parameters. First find average of each parameter. 

In [10]:
i = 0
block2_semifinal = {}
while i < len(animal_numbers):
    print(i)
    j = 0
    param_sum = {}
    block2_ave = {}
    while j < len(block2_param):
        param_sum[block2_param[j]] = 0
        for row in data[animal_numbers[i]][51:91]:
            if row[7] == block2_param[j]:
                param_sum[block2_param[j]] = param_sum[block2_param[j]] + np.int(row[11])
        ave = param_sum[block2_param[j]]/5
        block2_ave[block2_param[j]] = ave
        j = j + 1
    block2_semifinal[animal_numbers[i]] = block2_ave
    i = i + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


Loop through animal numbers to compute PPI for each block 2 parameter (using block 1 pulse data from first 5 trials)

In [11]:
i = 0
block2_final = {}
while i < len(animal_numbers):
    print(i)
    j = 0
    block2_ave = {}
    while j < len(block2_param):
        block2_ave[block2_param[j]] = np.float(block2_semifinal[animal_numbers[i]][block2_param[j]]) / np.float(block1ave_final[animal_numbers[i]][0])
        j = j + 1
    block2_final[animal_numbers[i]] = block2_ave
    i = i + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


Combine blocks with only one data point per animal into an array and put array into dictionary stored via animal

In [12]:
i = 0
aveblocks_final = {}
aveblocks = [twobreak_final]
while i < len(animal_numbers):
    print(i)
    j = 0
    ave_final = []
    while j < len(aveblocks):
        ave_final.append(aveblocks[j][animal_numbers[i]])
        j = j + 1
    aveblocks_final[animal_numbers[i]] = ave_final
    i = i + 1

NameError: name 'twobreak_final' is not defined

Create final lists of data and block names.

In [13]:
final = [aveblocks_final, block1ave_final, block2_final]
finalnames = ['aveblocks_final', 'block1ave_final', 'block2_final']

Use pandas to convert each final block data into a dataframe. Use ExcelWriter to export each data frame to a separate tab in an excel workboook.

In [14]:
import pandas as pd
writer = pd.ExcelWriter(root.filename + '.analyzed.xlsx', engine='xlsxwriter')
i = 0
for row in final:
    print(i)
    df = pd.DataFrame(data=row)
    df.to_excel(writer, sheet_name=finalnames[i])
    i = i + 1
writer.save()

0
1
2
